In [20]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import math

In [21]:
def read_timeseries_data(subnum, dataloc="../tseries", runtype="pair", runnum=1):
    fname = f"{dataloc}/sub-{subnum}_task-{runtype}_run-{runnum}_space-fsLR_den-91k_bold_timeseries.tsv"
    data = pd.read_csv(fname, sep="\t")
    return data

def read_metadata(subnum, dataloc="../tseries", runtype="pair", runnum=1):
    fname = f"{dataloc}/sub-{subnum}_task-{runtype}_run-0{runnum}_events.tsv"
    data = pd.read_csv(fname, sep="\t")
    return data

def find_image_pairs(subnum="01"):
    meta = read_metadata(subnum)
    image_pairs = meta["trial_type"].unique()
    return image_pairs

def reducing_metadata_pulling_timeseries(image_of_interest, subnum="01"):
    #creating a list for onsets
    onsets = []
    #making a weights dictionary, where:
    ##if the time is 0: we don't need to weight anything out, hence 1
    ##if the time is 0.5: we create a weighting that takes 2 thirds of the first TR and 1 third of the second
    ##if the time is 1: we create a weighting that takes 1 third of the first TR and 2 thirds of the second
    weights_dict = dict(zip([0, 0.5, 1], [[1], [2/3, 1/3], [1/3, 2/3]]))
    #read in the metadata
    meta = read_metadata(subnum)
    #read in the timeseries data
    tseries = read_timeseries_data(subnum)
    #added in: deleting the first row of the timeseries data, as it is the labels
    #tseries = tseries.iloc[1:]
    #we isolate the timeseries data to only the rows of the image we are interested in
    img_rows = meta[meta['trial_type'] == image_of_interest]
    #and extract those onset times from the condensed dataset prior and turn those numbers to a list
    onset_time = img_rows["onset"].to_list()
    #create a for loop for every onset time 
    appended_data = []
    for i in onset_time:
        #start a counter at 0
        counter = 0
        #take the onset time and turn it into TR, dividing by 1.5 and adding 3
        TR = (i/1.5) + 3
        #our var variable is equal to the onset time divided by 1.5
        var = i%1.5 
        #our new TR is the onset time rounded down
        TR = math.floor(TR)
        #if our var variable is 0, we just take the rows from the TR we set earlier
        #if our var variable is not 0, we will take 2 rows from the metadata, TR and TR +1 
        rows = [TR] if var == 0 else [TR,TR+1]
        #here, we define weights and use our var time to grab the corresponding weightings
        weights = weights_dict[var]
        #for every row in our meta data (either 1 or 2 rows) and every weight in our weights variable
        for row, weight in zip(rows, weights):
            #find the row in the timeseries data that corresponds to the row number and multiply it by the respective weights, either 1 if there is only 1 row, or if there are two rows it gets multiplied accordingly to the weights
            trow = tseries.iloc[int(row)] * weight 
            trow = pd.DataFrame(trow).transpose()
            #if the counter is 0, our trow is just our trow, but if not, we have to concat the 2 timeseries rows
            TROW = trow if counter == 0 else pd.concat([TROW, trow])
            #counter incrementing
            counter += 1
        #add rows together
        TROW = TROW.sum(axis=0).to_frame().transpose()
        #concatenate TROWS together
        appended_data.append(TROW)
    appended_data = pd.concat(appended_data)
    display(appended_data)

    
image = find_image_pairs()
reducing_metadata_pulling_timeseries(image_of_interest = image[0])  

,7Networks_LH_Vis_1,7Networks_LH_Vis_2,7Networks_LH_Vis_3,7Networks_LH_Vis_4,7Networks_LH_Vis_5,7Networks_LH_Vis_6,7Networks_LH_Vis_7,7Networks_LH_Vis_8,7Networks_LH_Vis_9,7Networks_LH_Vis_10,...,7Networks_RH_Default_pCunPCC_11,7Networks_RH_Default_pCunPCC_12,7Networks_RH_Default_pCunPCC_13,7Networks_RH_Default_pCunPCC_14,7Networks_RH_Default_pCunPCC_15,7Networks_RH_Default_pCunPCC_16,7Networks_RH_Default_pCunPCC_17,7Networks_RH_Default_pCunPCC_18,7Networks_RH_Cont_pCun_2,7Networks_RH_Cont_pCun_4
0,0.815905,-0.537616,-0.060012,-0.246175,-0.718148,-0.018797,-0.931194,-0.580945,0.252151,-0.442175,...,-0.909774,0.104488,-1.065234,1.591322,-0.514525,-0.197556,-0.242896,-0.162824,-0.367419,-1.111739
0,-0.345539,-0.688186,1.778215,0.318641,0.846800,0.233564,0.291422,0.716534,-0.169153,-0.236017,...,0.540716,-0.013586,0.909512,0.363607,-0.789495,0.261070,0.073796,0.378142,0.923575,-0.105384
0,0.218936,-1.004982,-0.212121,-0.925329,2.597476,-0.649049,-0.330054,-1.038484,-0.656959,-0.290501,...,-0.110236,-0.465160,0.017416,-0.368470,-0.531046,0.183877,0.512200,0.675188,0.539712,1.169043
0,1.111301,0.414443,-0.643596,0.438940,0.780574,-0.387592,-0.244199,0.354787,-0.286386,0.241504,...,-0.451732,-0.493007,0.762192,-0.464391,-0.270607,0.047441,1.387619,0.313966,-0.306827,1.149412
0,-0.480739,0.239341,-0.473650,-0.195432,0.569910,0.095572,-0.786097,-0.385514,-0.796777,0.618817,...,1.191373,1.414873,0.434431,0.812845,0.203357,0.255649,-0.134269,-1.364178,-0.259150,1.158107
